In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import warnings
import model_report as mr
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df_train = pd.read_csv('C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\train_comp_nouns_v1.csv')
df_test = pd.read_csv('C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\test_comp_nouns_v1.csv')
df_holdout = pd.read_csv('C:\\Users\\blgai\\OneDrive\\Documents\\School\\SMU\\Courses\\Fall 2021\\Capstone A\Data\\holdout_comp_nouns_v1.csv')

In [3]:
#training data
X_train = df_train['clean_comp_nouns']
y_train = df_train['category']

#test data
X_test = df_test['clean_comp_nouns']
y_test = df_test['category']

#holdout data
X_holdout = df_holdout['clean_comp_nouns']
y_holdout = df_holdout['category']

In [ ]:
from sklearn import metrics
def get_metrics(true_labels, predicted_labels):
    my_accuracy = np.round(metrics.accuracy_score(true_labels,predicted_labels),4)
    my_precision = np.round(metrics.precision_score(true_labels,predicted_labels,average='weighted'),4)
    my_TPR = np.round(metrics.recall_score(true_labels,predicted_labels,average='weighted'),4)
    my_F1 = np.round(metrics.f1_score(true_labels,predicted_labels,average='weighted'),4)
    
    return my_accuracy, my_precision, my_TPR, my_F1

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn_relief as relief
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

svm = LinearSVC(penalty='l2',C=1,class_weight={'dr':8,'sx':10},random_state=1234)

feat_select_k = [3200,3500,3800,4100,4400,4700,5000]

#build BOW features on train corpus
tv = TfidfVectorizer(use_idf=True, min_df=25,max_df=.9, norm="l2",smooth_idf=True)
tv_train_features = tv.fit_transform(X_train)
#transform test corpus into features
tv_test_features = tv.transform(X_test)
#transform holdout corpus into features
tv_holdout_features = tv.transform(X_holdout)

#create an object to collect metrics for comparison
data_dict = []

#select optimal features, train, test, record metrics
for i in feat_select_k:
    
    #find optimal features of size k
    r = relief.ReliefF(n_features=i)
    transformed_matrix = r.fit_transform(tv_train_features_array,np.array(y_train))
    #create a data frame that includes all features and whether or not they are deemed important by feature_selection
    feat_imp = pd.DataFrame(list(zip(tv.get_feature_names(),X_opt.get_support().tolist())),columns = ['Features','Important'])
    #Now we can grab the important features so that we can select only the important features from the sparse matrix created by TfidfVectorizer
    imp_Feats_array = np.array(feat_imp[feat_imp.Important == True].index)
    #Now we can pull only the important features out of the original train, test, and holdout matrices
    tv_train_features_sub = tv_train_features[:,imp_Feats_array]
    tv_test_features_sub = tv_test_features[:,imp_Feats_array]
    tv_holdout_features_sub = tv_holdout_features[:,imp_Feats_array]
    
    #train model using only the optimal features
    svm.fit(tv_train_features_sub,y_train)
    #get efficacy metrics of trained model
    svm_tfidf_cv_scores = cross_val_score(svm,tv_train_features_sub,y_train,cv=5)
    svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)
    svm_tfidf_test_score = svm.score(tv_test_features_sub,y_test)
    svm_predictions = svm.predict(tv_test_features_sub)
    
    accuracy, precision, tpr, f1 = get_metrics(true_labels=y_test,predicted_labels=svm_predictions)
    
    #store metrics in dictionary
    tmp_dict = {'No_Features':i,
                'Model':'SVM',
                'Class_Weights':'dr:8 and sx:10',
                'tfidf_mindf':25,
                'tfidf_maxdf':.9,
                'cv_5_mean_acc':svm_tfidf_cv_mean_score,
                'test_acc':accuracy,
                'precision':precision,
                'TPR/Recall':tpr,
                'F1 Score':f1
               }
    #append metrics from latest model to dictionary object
    data_dict.append(tmp_dict)

#create dataframe from dictionary object
df_overall = pd.DataFrame(data_dict)

#view all metrics
df_overall  


In [ ]:
tv_train_features_array = tv_train_features.toarray()
tv_test_features_array = tv_test_features.toarray()
tv_holdout_features_array = tv_holdout_features.toarray()

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

svm = LinearSVC(penalty='l2',C=1,class_weight={'dr':8,'sx':10},random_state=1234)

feat_select_k = [3200,3500,3800,4100,4400,4700,5000]

#build BOW features on train corpus
tv = TfidfVectorizer(use_idf=True, min_df=25,max_df=.9, norm="l2",smooth_idf=True)
tv_train_features = tv.fit_transform(X_train)
#transform test corpus into features
tv_test_features = tv.transform(X_test)
#transform holdout corpus into features
tv_holdout_features = tv.transform(X_holdout)

X_opt=SelectKBest(chi2, k=4100)
tv_train_features_trimmed = X_opt.fit_transform(tv_train_features, y_train)
tv_train_features_trimmed.shape

(29302, 4100)

In [6]:
#create a data frame that includes all features and whether or not they are deemed important by feature_selection
feat_imp = pd.DataFrame(list(zip(tv.get_feature_names(),X_opt.get_support().tolist())),columns = ['Features','Important'])

In [7]:
#Now we can grab the important features so that we can select only the important features from the sparse matrix created by TfidfVectorizer
imp_Feats_array = np.array(feat_imp[feat_imp.Important == True].index)

In [8]:
#Now we can pull only the important features out of the original train, test, and holdout matrices
tv_train_features_sub = tv_train_features[:,imp_Feats_array]
tv_test_features_sub = tv_test_features[:,imp_Feats_array]
tv_holdout_features_sub = tv_holdout_features[:,imp_Feats_array]

In [9]:
tv_train_features_sub_array = tv_train_features_sub.toarray()

In [ ]:
import sklearn_relief as relief
r = relief.ReliefF(n_features=50)
transformed_matrix = r.fit_transform(tv_train_features_sub_array,np.array(y_train))

In [ ]:
svm = LinearSVC(penalty='l2',C=1,class_weight={'dr':8,'sx':10},random_state=1234)
scm.fit(tv_train_features_sub,train_label)

In [ ]:
r = relief.ReliefF(n_features=50)
transformed_matrix = r.fit_transform(tv_train_features_array,np.array(y_train))

In [ ]:
shape(transformed_matrix)

In [ ]:
print(r.w_)

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', C=1.0, class_weight={'dr':8,'sx':10},random_state=1234)
tmp = sparse.csr_matrix(tv_train_features_array)
clf.fit(tmp,y_train)

In [ ]:
type(tv_train_features)

In [ ]:
type(tv_train_features_array)

In [ ]:
from scipy import sparse
type(sparse.csr_matrix(tv_train_features_array))